## Create Snow-On DEM with JSON Pipeline using PDAL

## Import Libraries

In [ ]:
import numpy as np
import pdal
import json
import laspy
import open3d as o3d
import os
import matplotlib.pyplot as plt
import rioxarray as rxa
from os.path import join, basename, exists, dirname, abspath
from glob import glob
import geopandas as gpd
import fiona
from zipfile import ZipFile
import shlex
import subprocess
import py3dep
from shapely.geometry import box
import pyproj
from shapely.geometry import Point
from shapely.ops import transform
import contextily as cx

## Define Functions
Right now we only have one to do command line calls. Probably can be put into a seperate functions script and imported if we want to clean this up.

In [ ]:
def cl_call(commamd):
    process = subprocess.Popen(commamd, 
                           stdout=subprocess.PIPE,
                           universal_newlines=True)

    while True:
        output = process.stdout.readline()
        print(output.strip())
        # Do something else
        return_code = process.poll()
        if return_code is not None:
            print('RETURN CODE', return_code)
            # Process has finished, read rest of the output 
            for output in process.stdout.readlines():
                print(output.strip())
            break

## Capture filepaths for .laz and .kmz

test_dir = directory name. We can iterate over this variable in future full runs to do all the lidar processing with one function

In [ ]:
test_dir = abspath('../test/09_EXPORT')
laz_fps = glob(join(test_dir,'*.laz'))
kmz_fp = glob(join(test_dir,'*.kmz'))[0]

## Extract and visualize helicopter flight path (not neccessary for lidar processing)

In [ ]:
with ZipFile(kmz_fp, 'r') as kmz:
    kml_fps = kmz.namelist()
    kmz.extractall(test_dir)
kml_fp = [fp for fp in kml_fps if fp.endswith('kml')][0]
my_map = gpd.read_file(join(test_dir, kml_fp), driver='KML')
my_map[::1000].explore()

## Mosaicing all LAZ files to one
Tried Las2las - really didn't work. Pdal merge seems to be working (based on file size alone)

In [ ]:
join(test_dir, '*.laz')


In [ ]:
test_dir

In [ ]:
' '.join(glob(join(test_dir, '*.laz')))

In [ ]:
las_prefix = '20220215'
test_dir = '/Users/zachkeskinen/Documents/ice-road-copters/test/Subset'
in_str = ' '.join(glob(join(test_dir, '*.laz')))
mosaic_fp = join(test_dir, 'merge.laz')
mosaic_cmd = f'pdal merge {in_str} {mosaic_fp}'
mosaic_cmd = shlex.split(mosaic_cmd)
print(mosaic_cmd)
cl_call(mosaic_cmd)

## Download DEM for dem filtering step

Needs to be projected into the right CRS (got from las files)

In [ ]:
import laspy

with laspy.open(mosaic_fp) as data:
     hdr = data.header
     crs = hdr.parse_crs()
wgs84 = pyproj.CRS('EPSG:4326')
utm_bounds = box(hdr.mins[0], hdr.mins[1], hdr.maxs[0], hdr.maxs[1])
project = pyproj.Transformer.from_crs(crs, wgs84 , always_xy=True).transform
wgs84_bounds = transform(project, utm_bounds)
dem_wgs = py3dep.get_map('DEM', wgs84_bounds, resolution = 1, crs = 'epsg:4326')
dem_utm = dem_wgs.rio.reproject(crs)
dem_fp = join('/Users/zachkeskinen/Documents/ice-road-copters/test/dems','full.tif')
dem_utm.rio.to_raster(dem_fp)
print(crs)
print(crs.ellipsoid)

## Generate outfile names for JSON

can also be used with:
- --readers.las.filename=str(out_fp)
- --writers.las.filename=str(join(test_dir, 'out.laz'))
- --writers.gdal.filename=str(join(test_dir, 'out.tif'))

In [ ]:
mosaic_fp = join(test_dir, 'merge.laz')
outlas = str(join(test_dir, 'outv3.laz'))
outtif = str(join(test_dir, 'outv3.tif'))
json_dir = './jsons'
run_number = 1
run_name = 'testing'
json_to_use = './jsons/las2dem-dom.json'

## Create JSON

In [ ]:
import json
# good docs on types of filters used: https://pdal.io/stages/filters.html#ground-unclassified
# Reads in mosaiced las file
reader = {"type": "readers.las", "filename": mosaic_fp}

# Filters out points with 0 returns
mongo_filter = {"type": "filters.mongo",\
    "expression": {"$and": [\
        {"ReturnNumber": {"$gt": 0}},\
             {"NumberOfReturns": {"$gt": 0}} ] } }
# Filter out points far away from our dem
dem_filter = {
        "type":"filters.dem",
        "raster":dem_fp,
        "limits":"Z[25:35]"
    }
# Extended Local Minimum filter ()
elm_filter = {"type": "filters.elm"}

outlier_filter = {"type": "filters.outlier",\
    "method": "statistical",\
        "mean_k": 12,\
            "multiplier": 2.2}

smrf_classifier = {"type": "filters.smrf",\
    "ignore": "Classification[7:7], NumberOfReturns[0:0], ReturnNumber[0:0]"}

smrf_selecter = { 
        "type":"filters.range",
        "limits":"Classification[2:2]"
    }

las_writer = {"type": "writers.las",\
#     "where": "Classification[2:2]",\
        "filename":outlas}

tif_writer = {"type": "writers.gdal",\
#     "where": "Classification[2:2]",\
        "filename":outtif,
        "resolution":1.0,
        "output_type":"idw"}

pipeline = [reader, mongo_filter, dem_filter, elm_filter, outlier_filter, smrf_classifier,smrf_selecter, las_writer, tif_writer]

os.makedirs(json_dir, exist_ok= True)
json_to_use = join(json_dir, f'{run_name}-r{run_number}.json')
with open(json_to_use,'w') as outfile:
    json.dump(pipeline, outfile, indent = 2)

## Running JSON Pipeline

In [ ]:
# pipeline_cmd = f'pdal pipeline -i {json_to_use} --readers.las.filename="{mosaic_fp}" --writers.las.filename={outlas} --writers.gdal.filename={outtif} -v 8'
pipeline_cmd = f'pdal pipeline -i {json_to_use} --readers.las.filename="{laz_fps[6]}" -v 8'
pipeline_cmd = shlex.split(pipeline_cmd)
print(pipeline_cmd)
cl_call(pipeline_cmd)

## Visualization

o3d visualization is awesome but seems to kill my kernel everytime when I try and close the window. 

Rioxarray visualization seems safer for quick checks

In [ ]:
import numpy as np
import pdal
import json
import laspy
import open3d as o3d
import os
import matplotlib.pyplot as plt
import rioxarray as rxa
from os.path import join, basename, exists, dirname, abspath
from glob import glob
import geopandas as gpd
import fiona
from zipfile import ZipFile
import shlex
import subprocess
import py3dep
from shapely.geometry import box
import pyproj
from shapely.geometry import Point
from shapely.ops import transform
import contextily as cx

In [ ]:
import laspy
import numpy as np
PC = laspy.read('/Users/zachkeskinen/Documents/ice-road-copters/test/mcs-subset/ice-road/results/clipped_PC.laz')

points = np.vstack((PC.X, PC.Y, PC.Z)).transpose()

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
#pcd.colors = o3d.utility.Vector3dVector(colors/65535)
#pcd.normals = o3d.utility.Vector3dVector(normals)0
o3d.visualization.draw_geometries([cloud])